<a href="https://colab.research.google.com/github/AncalagonDBlack/source-code-seo-tool/blob/master/keyword_cannibalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qgrid
!pip install build
#!pip install jupyter

import pandas as pd
import re
import datetime
import qgrid
from collections import defaultdict
from dateutil import relativedelta
import httplib2
from apiclient import errors
from apiclient.discovery import build
from oauth2client.client import OAuth2WebServerFlow
import requests
from bs4 import BeautifulSoup
from google.colab import output
from google.colab import drive
output.enable_custom_widget_manager()
pd.set_option('display.max_columns', None)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 889 kB 7.8 MB/s 
  Created wheel for qgrid: filename=qgrid-1.3.1-py2.py3-none-any.whl size=1761272 sha256=d948edc83d3a2136da6895a7cfd0fdae42348aebafa3b9b2a8a7923486e4c107
  Stored in directory: /root/.cache/pip/wheels/53/87/cb/c63a58774627471976061afb5b8657b76d2268c1e76e980586
Successfully built qgrid
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
CLIENT_ID = '466919438491-f847p82hb2dchtdjblcneu34qsdd1aqb.apps.googleusercontent.com'
CLIENT_SECRET = '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'
site = 'https://bbcincorp.com'

In [ ]:
end_date = datetime.date.today()
#end_date = datetime.date.today()-relativedelta.relativedelta(days=3) #My Note
start_date = end_date - relativedelta.relativedelta(months=6)

OAUTH_SCOPE = 'https://www.googleapis.com/auth/webmasters.readonly'
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

In [ ]:
# Run through the OAuth flow and retrieve credentials
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, redirect_uri=REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print ('Go to the following link in your browser: ' + authorize_url)
code = input('Enter verification code: ').strip()
credentials = flow.step2_exchange(code)

Go to the following link in your browser: https://accounts.google.com/o/oauth2/v2/auth?client_id=466919438491-f847p82hb2dchtdjblcneu34qsdd1aqb.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly&access_type=offline&response_type=code
Enter verification code: 4/1AdQt8qgBwceTCFM9IdQkgQ4cO2MfvdJLQe9abuGf2o8u9c62X5Vu3qg9aVE


In [ ]:
# Create an httplib2.Http object and authorize it with our credentials
http = httplib2.Http()
http = credentials.authorize(http)
 
webmasters_service = build('searchconsole', 'v1', http=http)

def execute_request(service, property_uri, request):
    return service.searchanalytics().query(siteUrl=property_uri, body=request).execute()
#dict 
request = {
    'startDate': datetime.datetime.strftime(start_date,"%Y-%m-%d"),
    'endDate': datetime.datetime.strftime(end_date,'%Y-%m-%d'),
    'dimensions': ['page','query','country'], #list
    'rowLimit': 20000 #up to 20.000 urls giới hạn đó
}

In [ ]:
#Adding a device filter to request
device_category = input('Enter device category: MOBILE, DESKTOP or TABLET (leave it blank for all devices): ').strip()
if device_category:
    request['dimensionFilterGroups'] = [{'filters':[{'dimension':'device','expression':device_category}]}]

Enter device category: MOBILE, DESKTOP or TABLET (leave it blank for all devices): 


In [ ]:
#Request to SC API
response = execute_request(webmasters_service, site, request)

In [ ]:
#Parse the JSON Returned
scDict = defaultdict(list)
 
for row in response['rows']:
    scDict['page'].append(row['keys'][0] or 0)
    scDict['query'].append(row['keys'][1] or 0)
    scDict['country'].append(row['keys'][2] or 0)
    scDict['clicks'].append(row['clicks'] or 0)
    scDict['ctr'].append(row['ctr'] or 0)
    scDict['impressions'].append(row['impressions'] or 0)
    scDict['position'].append(row['position'] or 0)

In [ ]:
#Create a DataFrame With the Google Search Console Data
df = pd.DataFrame(data = scDict)
 
df['clicks'] = df['clicks'].astype('int')
df['ctr'] = df['ctr']*100
df['impressions'] = df['impressions'].astype('int')
df['position'] = df['position'].round(2)
df['country']=df['country'].astype('str')
df.sort_values('clicks',inplace=True,ascending=False)
df

,page,query,country,clicks,ctr,impressions,position
0,https://bbcincorp.com/resources/differences-be...,difference between tax planning tax avoidance ...,ind,494,51.298027,963,1.05
1,https://bbcincorp.com/,bbcincorp,vnm,346,49.927850,693,2.22
2,https://bbcincorp.com/resources/countries-with...,best offshore bank accounts,usa,320,21.404682,1495,1.97
3,https://bbcincorp.com/resources/countries-with...,offshore bank account,usa,310,3.288427,9427,6.86
4,https://bbcincorp.com/resources/countries-with...,tax free countries,gbr,287,6.016771,4770,4.84
...,...,...,...,...,...,...,...
10304,https://bbcincorp.com/resources/a-guide-to-bvi...,bvi economic substance filing deadline 2022,twn,1,100.000000,1,2.00
10305,https://bbcincorp.com/resources/a-guide-to-bvi...,bvi economic substance filing fee,hkg,1,2.857143,35,4.66
10306,https://bbcincorp.com/resources/a-guide-to-bvi...,bvi economic substance filing fee,sgp,1,14.285714,7,4.29
10307,https://bbcincorp.com/resources/a-guide-to-bvi...,bvi economic substance financial period,hkg,1,14.285714,7,5.29


In [ ]:
#Clean the DataFrame and Sort by Query
SERP_results = 8 #insert here your prefered value for SERP results
#branded_queries = 'brand|vrand|b rand...' #insert here your branded queries

branded_queries = 'bbcincorp|bbc|bbc incorp' #insert here your branded queries 
df_canibalized = df[df['position'] > SERP_results] 
df_canibalized = df_canibalized[~df_canibalized['query'].str.contains(branded_queries, regex=True)]
df_canibalized = df_canibalized[df_canibalized.duplicated(subset=['query'], keep=False)]
df_canibalized.set_index(['query'],inplace=True)
df_canibalized.sort_index(inplace=True)
df_canibalized.reset_index(inplace=True)
df_canibalized

,query,page,country,clicks,ctr,impressions,position
0,0 corporate tax countries,https://bbcincorp.com/resources/countries-with...,esp,1,16.666667,6,10.17
1,0 corporate tax countries,https://bbcincorp.com/resources/countries-with...,zaf,1,33.333333,3,10.67
2,account offshore,https://bbcincorp.com/resources/countries-with...,can,1,5.555556,18,8.44
3,account offshore,https://bbcincorp.com/resources/countries-with...,fra,1,6.250000,16,8.31
4,accounting services hong kong,https://bbcincorp.com/hong-kong-company-format...,are,1,11.111111,9,10.78
...,...,...,...,...,...,...,...
2872,zero tax countries,https://bbcincorp.com/resources/countries-with...,sgp,1,4.545455,22,8.64
2873,zero tax countries,https://bbcincorp.com/resources/countries-with...,ind,2,1.010101,198,10.23
2874,zero tax countries,https://bbcincorp.com/resources/countries-with...,egy,1,6.250000,16,9.62
2875,الفرق بين offshore و onshore,https://bbcincorp.com/resources/onshore-vs-off...,sau,1,0.632911,158,8.92


In [ ]:
#Scrape URLs and Add Titles and Meta Descriptions
def get_meta(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    title = soup.find('title').get_text()
    meta = soup.select('meta[name="description"]')[0].attrs["content"] 
    return title, meta
 
df_canibalized['title'],df_canibalized['meta'] = zip(*df_canibalized['page'].apply(get_meta))

df_canibalized

IndexError: ignored

In [ ]:
#Visualize Data in Jupyter Notebook
grid = qgrid.show_grid(df_canibalized, show_toolbar=True)
grid

In [ ]:
drive.mount('/drive')
df_canibalized.to_csv('Cannibalization.csv')




Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
